<a href="https://colab.research.google.com/github/valerio-unifei/ECAA07/blob/main/ECAA07_06_Servidor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando o MySQL no Colab

In [ ]:
!apt-get install mysql-server > /dev/null
!service mysql start

 * Starting MySQL database server mysqld
No directory, logging in with HOME=/
   ...done.


Permitindo o acesso ao root

In [ ]:
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'"

## Instalando o PyMySQL para executar comandos SQL como código no Python

In [ ]:
!pip -q install PyMySQL
%load_ext sql
%config SqlMagic.feedback=True
%config SqlMagic.autopandas=True
%sql mysql+pymysql://root:root@/

     |████████████████████████████████| 43 kB 1.1 MB/s 


'Connected: root@'

# Criando Banco de Dados

Base Exemplo:

https://github.com/datacharmer/test_db/blob/master/employees.sql

In [ ]:
%sql CREATE DATABASE ECAA07Base;

 * mysql+pymysql://root:***@/
1 rows affected.


""


## Direcionando base para criação de tabelas e dados

In [ ]:
%sql USE ECAA07Base;

 * mysql+pymysql://root:***@/
0 rows affected.


""


##Criando Tabelas

In [ ]:
%%sql

CREATE TABLE employees (
    emp_no      INT             NOT NULL,
    birth_date  DATE            NOT NULL,
    first_name  VARCHAR(14)     NOT NULL,
    last_name   VARCHAR(16)     NOT NULL,
    gender      ENUM ('M','F')  NOT NULL,
    hire_date   DATE            NOT NULL,
    PRIMARY KEY (emp_no)
);

CREATE TABLE departments (
    dept_no     CHAR(4)         NOT NULL,
    dept_name   VARCHAR(40)     NOT NULL,
    PRIMARY KEY (dept_no),
    UNIQUE  KEY (dept_name)
);

CREATE TABLE dept_emp (
    emp_no      INT             NOT NULL,
    dept_no     CHAR(4)         NOT NULL,
    from_date   DATE            NOT NULL,
    to_date     DATE            NOT NULL,
    FOREIGN KEY (emp_no)  REFERENCES employees   (emp_no)  ON DELETE CASCADE,
    FOREIGN KEY (dept_no) REFERENCES departments (dept_no) ON DELETE CASCADE,
    PRIMARY KEY (emp_no,dept_no)
);

 * mysql+pymysql://root:***@/
0 rows affected.
0 rows affected.
0 rows affected.


""


## Inserindo valores

In [ ]:
%%sql

INSERT INTO employees
  VALUES
  (1,'1974-06-30','Jupiter','Natalino','M','2015-09-10'),
  (2,'1978-12-10','Amancia','Mefisto','F','2016-02-14'),
  (3,'1984-05-23','Mefisto','Papanatas','M','2016-02-14'),
  (4,'1977-03-03','Sigismunda','Moluco','F','2013-06-07');

INSERT INTO departments
  VALUES
  (1,'Diretoria'),
  (2,'Vendas');

INSERT INTO dept_emp
  VALUES
  (1,1,'2015-09-10','2099-01-01'),
  (2,1,'2016-02-14','2019-03-10'),
  (3,2,'2016-02-14','2017-06-20'),
  (4,2,'2013-06-07','2099-01-01');

 * mysql+pymysql://root:***@/
4 rows affected.
2 rows affected.
4 rows affected.


""


## Definindo Usuários para Banco

In [ ]:
%%sql

CREATE USER 'ecaa07user'@'localhost' IDENTIFIED BY 'sqlehdivertido2021sqn';

GRANT ALL PRIVILEGES ON ECAA07Base.* TO 'ecaa07user'@'localhost';

CREATE USER 'ecaa07web'@'%' IDENTIFIED BY 'se#for#facil#a#turma&invade09';

GRANT SELECT ON ECAA07Base.employees TO 'ecaa07web'@'%';

 * mysql+pymysql://root:***@/
0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.


""


# Acesso Teste

Código 1

In [ ]:
import pymysql.cursors

conn = pymysql.connect(
    host="localhost",
    user="ecaa07user",
    password="sqlehdivertido2021sqn",
    database='ECAA07Base',
    )

cursor = conn.cursor()

cursor.execute('''
  SELECT
    e.first_name as nome,
    e.last_name as sobrenome,
    e.birth_date as nascimento,
    e.gender as sexo,
    e.hire_date as contratacao,
    d.dept_name as departamento
  FROM employees as e INNER JOIN dept_emp as de ON e.emp_no = de.emp_no
    INNER JOIN departments as d ON d.dept_no = de.dept_no
  '''
 )

cursor.fetchall()

(('Jupiter',
  'Natalino',
  datetime.date(1974, 6, 30),
  'M',
  datetime.date(2015, 9, 10),
  'Diretoria'),
 ('Amancia',
  'Mefisto',
  datetime.date(1978, 12, 10),
  'F',
  datetime.date(2016, 2, 14),
  'Diretoria'),
 ('Mefisto',
  'Papanatas',
  datetime.date(1984, 5, 23),
  'M',
  datetime.date(2016, 2, 14),
  'Vendas'),
 ('Sigismunda',
  'Moluco',
  datetime.date(1977, 3, 3),
  'F',
  datetime.date(2013, 6, 7),
  'Vendas'))

Código 2

In [ ]:
import pymysql.cursors

conn = pymysql.connect(
    host="localhost",
    user="ecaa07web",
    password="se#for#facil#a#turma&invade09",
    database='ECAA07Base',
    )

cursor = conn.cursor()

cursor.execute('''
  SELECT
    e.first_name as nome,
    e.last_name as sobrenome,
    e.birth_date as nascimento,
    e.gender as sexo,
    e.hire_date as contratacao,
    d.dept_name as departamento
  FROM employees as e INNER JOIN dept_emp as de ON e.emp_no = de.emp_no
    INNER JOIN departments as d ON d.dept_no = de.dept_no
  '''
 )

cursor.fetchall()

(('Jupiter',
  'Natalino',
  datetime.date(1974, 6, 30),
  'M',
  datetime.date(2015, 9, 10),
  'Diretoria'),
 ('Amancia',
  'Mefisto',
  datetime.date(1978, 12, 10),
  'F',
  datetime.date(2016, 2, 14),
  'Diretoria'),
 ('Mefisto',
  'Papanatas',
  datetime.date(1984, 5, 23),
  'M',
  datetime.date(2016, 2, 14),
  'Vendas'),
 ('Sigismunda',
  'Moluco',
  datetime.date(1977, 3, 3),
  'F',
  datetime.date(2013, 6, 7),
  'Vendas'))

In [ ]:
%%sql
GRANT SELECT ON ECAA07Base.dept_emp TO 'ecaa07web'@'%';
GRANT SELECT ON ECAA07Base.departments TO 'ecaa07web'@'%';

 * mysql+pymysql://root:***@/
0 rows affected.
0 rows affected.


""


Código 3

In [ ]:
import pymysql.cursors

conn = pymysql.connect(
    host="localhost",
    user="ecaa07web",
    password="se#for#facil#a#turma&invade09",
    database='ECAA07Base',
    )

cursor = conn.cursor()

cursor.execute(' SELECT * FROM employees')

cursor.fetchall()

((1,
  datetime.date(1974, 6, 30),
  'Jupiter',
  'Natalino',
  'M',
  datetime.date(2015, 9, 10)),
 (2,
  datetime.date(1978, 12, 10),
  'Amancia',
  'Mefisto',
  'F',
  datetime.date(2016, 2, 14)),
 (3,
  datetime.date(1984, 5, 23),
  'Mefisto',
  'Papanatas',
  'M',
  datetime.date(2016, 2, 14)),
 (4,
  datetime.date(1977, 3, 3),
  'Sigismunda',
  'Moluco',
  'F',
  datetime.date(2013, 6, 7)))

# Atividades

In [ ]:
# garanta o direito de criar tabelas para o usuário ecaa07web
%sql